Notebook purpose: evaluate how efficiently we could search for catalysts using the ML model under various constraints.

The most conspicuous constraint is to find a set number of active catalysts without any unnecessary DFT calculations
What is unnecessary? --> 100% of O2 binding calculations are to actual binding sites
So we can accept a model with lower accuracy as long as it has no false positives --> only a small penalty for false negatives

Let's say we're only willing to run 5 DFT O2 binding calculations, and we want basically all of them to show that we found active sites. We'd probably want each of these to be per catalyst, to show that we've found 5 unique active catalysts. Assuming we're working with 10% of the data as a "test" set, that's about 27 calalysts, so we want to pick the ones that the model is most confident have at least 1 site that binds O2.

Really, this is a question of whether the active sites for a set of catalysts are most likely to actually be binding
Can order by log-loss and take that as an estimate of uncertainty (is that a fair expectation?)


In [3]:
import pandas as pd
import numpy as np

# Preprocessing
from sklearn.model_selection import GroupShuffleSplit

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix

In [4]:
from ngcc_ml import data_tools
from ngcc_ml import skl_tools

RDKit WARNING: [14:00:41] Enabling RDKit 2019.09.3 jupyter extensions


In [5]:
df = pd.read_csv("/home/nricke/work/ngcc_ml/DidItBindv5.csv")
df["Doesitbind"] = df["Doesitbind"].astype("int")

In [6]:
df.columns

Index(['Unnamed: 0', 'Atom Number', 'Catalyst Name', 'CatalystO2File',
       'Element', 'SpinDensity', 'ChElPGPositiveCharge', 'ChElPGNeutralCharge',
       'ChargeDifference', 'Doesitbind', 'BondLength', 'IonizedFreeEnergy',
       'IonizationEnergy', 'BindingEnergy', 'NeutralFreeEnergy', 'OrthoOrPara',
       'Meta', 'FartherThanPara', 'DistanceToN', 'AverageBondLength',
       'BondLengthRange', 'NumberOfHydrogens', 'AromaticSize', 'IsInRingSize6',
       'IsInRingSize5', 'NeighborSpinDensity', 'NeighborChElPGCharge',
       'NeighborChargeDifference', 'AromaticExtent', 'RingEdge',
       'NumNitrogens', 'NumHeteroatoms', 'ring_nitrogens',
       'atom_plane_deviation', 'ring_plane_deviation', 'charge'],
      dtype='object')

In [23]:
df.shape

(4141, 36)

In [7]:
print(len(df["Catalyst Name"].unique()))
print(len(df[df["Doesitbind"] == 1]["Catalyst Name"].unique()))

267
250


In [8]:
feature_cols = {"SpinDensity", "ChElPGNeutralCharge", "ChargeDifference", "IonizationEnergy", "OrthoOrPara", "Meta", "FartherThanPara", "DistanceToN", "AverageBondLength",  "NumberOfHydrogens", "IsInRingSize6", "IsInRingSize5", "NeighborSpinDensity", 'NeighborChElPGCharge', 'NeighborChargeDifference', "AromaticExtent", "RingEdge", "NumNitrogens", "NumHeteroatoms", "charge", "atom_plane_deviation", "ring_plane_deviation", "ring_nitrogens"}
not_scaled_cols = {"OrthoOrPara", "Meta", "FartherThanPara", "NumberOfHydrogens", "IsInRingSize6", "IsInRingSize5", "RingEdge", "NumNitrogens", "NumHeteroatoms", "ring_nitrogens", "charge"}
df_scale = data_tools.process_data(df, scaledCols=list(feature_cols - not_scaled_cols))
train_inds, test_inds = next(GroupShuffleSplit(test_size=0.10, n_splits=2, random_state = 6).split(df, groups=df['Catalyst Name']))
train = df.iloc[train_inds]
test = df.iloc[test_inds]
X_train_group = train[feature_cols]
y_train_group = train["Doesitbind"]
X_test_group = test[feature_cols]
y_test_group = test["Doesitbind"]

/home/nricke/local/opt/miniconda/envs/ds/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/nricke/local/opt/miniconda/envs/ds/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [31]:
rfc = RandomForestClassifier(n_estimators=1000, max_depth=100, class_weight={0:1, 1:10})
rfc.fit(X_train_group, y_train_group)
print('Accuracy of RFC on test set: {:.3f}'.format(rfc.score(X_test_group, y_test_group)))
print('Accuracy of RFC on training set: {:.3f}'.format(rfc.score(X_train_group, y_train_group)))
y_pred_group = rfc.predict(X_test_group)
print(confusion_matrix(y_test_group, y_pred_group))

Accuracy of RFC on test set: 0.956
Accuracy of RFC on training set: 1.000
[[354   6]
 [ 13  60]]


In [32]:
p = rfc.predict_proba(X_test_group)
test = test.assign(predict_proba=p[:,1], prediction=y_pred_group)
test_sort = test.sort_values(by="predict_proba", ascending=False)[["Catalyst Name", "Doesitbind", "prediction", "predict_proba"]]
first_false = list(test_sort["Doesitbind"]).index(0)
print(len(test_sort.head(first_false)["Catalyst Name"].unique()))

21


In [29]:
test

,Unnamed: 0,Atom Number,Catalyst Name,CatalystO2File,Element,SpinDensity,ChElPGPositiveCharge,ChElPGNeutralCharge,ChargeDifference,Doesitbind,...,AromaticExtent,RingEdge,NumNitrogens,NumHeteroatoms,ring_nitrogens,atom_plane_deviation,ring_plane_deviation,charge,predict_proba,prediction
64,64,1,sf103x0,None,C,-0.024369,-0.402277,-0.359844,0.042433,False,...,0,0,2,2,0,0.000000e+00,0.000000e+00,0,0.001000,0
65,65,3,sf103x0,sf103x0O2-2_optsp_a0m2.out,C,0.133525,-0.327030,-0.488787,-0.161757,True,...,16,2,2,2,2,2.090000e-08,2.616000e-07,0,0.892000,1
66,66,4,sf103x0,None,C,-0.011304,0.297350,0.293939,-0.003411,False,...,16,1,2,2,2,3.932000e-07,2.616000e-07,0,0.001000,0
67,67,5,sf103x0,None,C,0.054394,-0.247035,-0.271989,-0.024954,False,...,16,2,2,2,2,2.325000e-07,2.616000e-07,0,0.006000,0
68,68,6,sf103x0,sf103x0O2-5_optsp_a0m2.out,C,-0.039489,-0.131948,-0.182671,-0.050723,False,...,16,2,2,2,2,1.073000e-07,2.616000e-07,0,0.036000,0
69,69,7,sf103x0,None,C,0.123069,-0.198786,-0.215525,-0.016739,False,...,16,1,2,2,2,4.711000e-07,2.616000e-07,0,0.002000,0
70,70,8,sf103x0,sf103x0O2-7_optsp_a0m2.out,C,-0.062347,0.328233,0.281964,-0.046269,True,...,16,2,2,2,2,8.877000e-07,2.616000e-07,0,0.764952,1
71,71,10,sf103x0,sf103x0O2-9_optsp_a0m2.out,C,-0.102982,0.338841,0.309317,-0.029524,True,...,16,2,2,2,2,6.473000e-07,2.616000e-07,0,0.770000,1
72,72,11,sf103x0,None,C,0.164427,-0.198088,-0.229619,-0.031531,False,...,16,1,2,2,2,3.080000e-08,2.616000e-07,0,0.004000,0
73,73,12,sf103x0,None,C,-0.087159,0.373565,0.346701,-0.026864,False,...,16,1,2,2,2,2.460000e-07,2.616000e-07,0,0.016000,0


In [37]:
all_scores, best_catalysts, total_sites, first_false_list, false_proba_list = [], [], [], [], []
test_dfs = []
group_col = "Catalyst Name"
target_col = "Doesitbind"
model = RandomForestClassifier(n_estimators=1000, max_depth=100, class_weight={0:1, 1:10})
split_groups = GroupShuffleSplit(test_size=0.10, n_splits=10).split(df, groups=df[group_col])
for train_inds, test_inds in split_groups:
    train = df.iloc[train_inds]
    test = df.iloc[test_inds]
    X_train_group = train[feature_cols]
    X_test_group = test[feature_cols]
    y_test_group = test[target_col]
    y_train_group = train[target_col]
    model.fit(X_train_group, y_train_group)
    score = model.score(X_test_group, y_test_group)
    all_scores.append(score)
    print('Accuracy of RFC on test set: {:.2f}'.format(score))
    print('Accuracy of RFC on training set: {:.2f}'.format(model.score(X_train_group, y_train_group)))
    y_pred_group = model.predict(X_test_group)
    print(confusion_matrix(y_test_group, y_pred_group))
    p = model.predict_proba(X_test_group)
    test = test.assign(predict_proba=p[:,1], prediction=y_pred_group)
    test_dfs.append(test)
    test_sort = test.sort_values(by="predict_proba", ascending=False)[["Catalyst Name", "Doesitbind", "prediction", "predict_proba"]]
    first_false = list(test_sort["Doesitbind"]).index(0)
    first_false_list.append(first_false)
    best_catalysts.append(len(test_sort.head(first_false)["Catalyst Name"].unique()))
    total_sites.append(test_sort.shape[0]) # first false is 0 indexed. If the first false is the 10th place, the value is 9, so this index is the same as the number of catalysts before the first false
    false_proba_list.append(test_sort.iloc[first_false]["predict_proba"])
print("mean:", np.mean(all_scores))
print("mean:", np.mean(best_catalysts))
print("mean:", np.mean(total_sites))

Accuracy of RFC on test set: 0.97
Accuracy of RFC on training set: 1.00
[[322   5]
 [  6  61]]
Accuracy of RFC on test set: 0.96
Accuracy of RFC on training set: 1.00
[[349   3]
 [ 15  66]]
Accuracy of RFC on test set: 0.96
Accuracy of RFC on training set: 1.00
[[331   7]
 [  9  60]]
Accuracy of RFC on test set: 0.96
Accuracy of RFC on training set: 1.00
[[312   1]
 [ 16  69]]
Accuracy of RFC on test set: 0.96
Accuracy of RFC on training set: 1.00
[[355   2]
 [ 14  56]]
Accuracy of RFC on test set: 0.93
Accuracy of RFC on training set: 1.00
[[356   8]
 [ 21  61]]
Accuracy of RFC on test set: 0.96
Accuracy of RFC on training set: 1.00
[[340   7]
 [  8  64]]
Accuracy of RFC on test set: 0.95
Accuracy of RFC on training set: 1.00
[[350   8]
 [ 14  63]]
Accuracy of RFC on test set: 0.97
Accuracy of RFC on training set: 1.00
[[340   3]
 [  8  54]]
Accuracy of RFC on test set: 0.94
Accuracy of RFC on training set: 1.00
[[362   4]
 [ 21  50]]
mean: 0.957524905767368
mean: 15.1
mean: 420.1


In [19]:
all_scores, best_catalysts, total_sites, first_false_list, false_proba_list = [], [], [], [], []
test_dfs = []
group_col = "Catalyst Name"
target_col = "Doesitbind"
model = RandomForestClassifier(n_estimators=1000, max_depth=100, class_weight={0:1, 1:10})
split_groups = GroupShuffleSplit(test_size=0.10, n_splits=10).split(df, groups=df[group_col])
for train_inds, test_inds in split_groups:
    train = df.iloc[train_inds]
    test = df.iloc[test_inds]
    X_train_group = train[feature_cols]
    X_test_group = test[feature_cols]
    y_test_group = test[target_col]
    y_train_group = train[target_col]
    model.fit(X_train_group, y_train_group)
    score = model.score(X_test_group, y_test_group)
    all_scores.append(score)
    print('Accuracy of RFC on test set: {:.2f}'.format(score))
    print('Accuracy of RFC on training set: {:.2f}'.format(model.score(X_train_group, y_train_group)))
    y_pred_group = model.predict(X_test_group)
    print(confusion_matrix(y_test_group, y_pred_group))
    p = model.predict_proba(X_test_group)
    test = test.assign(predict_proba=p[:,1], prediction=y_pred_group)
    test_dfs.append(test)
    test_sort = test.sort_values(by="predict_proba", ascending=False)[["Catalyst Name", "Doesitbind", "prediction", "predict_proba"]]
    first_false = list(test_sort["Doesitbind"]).index(0)
    first_false_list.append(first_false)
    best_catalysts.append(len(test_sort.head(first_false)["Catalyst Name"].unique()))
    total_sites.append(test_sort.shape[0]) # first false is 0 indexed. If the first false is the 10th place, the value is 9, so this index is the same as the number of catalysts before the first false
    false_proba_list.append(test_sort.iloc[first_false]["predict_proba"])
print("mean:", np.mean(all_scores))
print("mean:", np.mean(best_catalysts))
print("mean:", np.mean(total_sites))

Accuracy of RFC on test set: 0.96
Accuracy of RFC on training set: 1.00
[[352   5]
 [ 12  64]]
Accuracy of RFC on test set: 0.93
Accuracy of RFC on training set: 1.00
[[332   4]
 [ 25  56]]
Accuracy of RFC on test set: 0.93
Accuracy of RFC on training set: 1.00
[[325   2]
 [ 29  64]]
Accuracy of RFC on test set: 0.94
Accuracy of RFC on training set: 1.00
[[334   5]
 [ 19  59]]
Accuracy of RFC on test set: 0.94
Accuracy of RFC on training set: 1.00
[[352   3]
 [ 21  54]]
Accuracy of RFC on test set: 0.98
Accuracy of RFC on training set: 1.00
[[300   4]
 [  5  62]]
Accuracy of RFC on test set: 0.96
Accuracy of RFC on training set: 1.00
[[384   4]
 [ 16  57]]
Accuracy of RFC on test set: 0.96
Accuracy of RFC on training set: 1.00
[[316   5]
 [ 12  77]]
Accuracy of RFC on test set: 0.93
Accuracy of RFC on training set: 1.00
[[269   1]
 [ 25  84]]
Accuracy of RFC on test set: 0.94
Accuracy of RFC on training set: 1.00
[[332   5]
 [ 21  57]]
mean: 0.9463658923177952
mean: 19.5
mean: 415.3


In [21]:
df_test_all = pd.concat(test_dfs)

In [42]:
for measure_list in [best_catalysts, false_proba_list, first_false_list]:
    print(np.mean(measure_list), np.min(measure_list), np.max(measure_list))

15.1 3 25
0.8897947576537207 0.557 0.994
25.7 4 53


Now that we've seen this is relatively successful in this framework, the next step is to do a head-to-head search comparison.
For a set of C catalysts, search until a subset A are found that are active, with the goal of checking O2 binding for as few as possible.
This is really quite similar to above, but we just want to keep track of slightly different metrics. For each group, we now want to instead ask 

In [30]:
def search_for_active_catalysts(df_catalysts, order_col, feature_cols, target_col="Doesitbind", find_num=10):
    """
    df_catalysts (pandas dataframe): catalysts to search
    order_col (str): column name to sort catalysts by. Expected for predict_proba or random values
    """
    df_sort = df_catalysts.sort_values(by=order_col, ascending=False)
    found_list = []
    count = 0
    for index, row in df_sort.iterrows():
        if row["Catalyst Name"] not in found_list:
            if row[target_col] == 1:
                found_list.append(row["Catalyst Name"])
            count += 1
            assert len(found_list) <= find_num
            if len(found_list) == find_num:
                break
    return found_list, count

In [9]:
df_ts = df.copy()
df_ts

,Unnamed: 0,Atom Number,Catalyst Name,CatalystO2File,Element,SpinDensity,ChElPGPositiveCharge,ChElPGNeutralCharge,ChargeDifference,Doesitbind,...,NeighborChElPGCharge,NeighborChargeDifference,AromaticExtent,RingEdge,NumNitrogens,NumHeteroatoms,ring_nitrogens,atom_plane_deviation,ring_plane_deviation,charge
0,0,1,sf100x0,None,C,-0.008245,-0.275350,-0.200227,0.075123,0,...,0.363881,-0.176308,0,0,1,5,0,0.000000e+00,0.000000,0
1,1,3,sf100x0,sf100x0O2-2_optsp_a0m2.out,C,0.555664,-0.064043,-0.339572,-0.275529,1,...,0.338356,-0.005618,18,2,1,5,1,6.723311e-02,0.263086,0
2,2,4,sf100x0,None,C,-0.181519,0.037008,0.096249,0.059241,0,...,-0.680474,-0.444099,18,1,1,5,1,3.875031e-02,0.263086,0
3,3,5,sf100x0,None,C,0.208580,-0.226453,-0.308850,-0.082397,0,...,0.449458,0.030624,18,2,1,5,1,1.263070e-05,0.263086,0
4,4,6,sf100x0,sf100x0O2-5_optsp_a0m2.out,C,-0.119560,0.176015,0.163894,-0.012121,0,...,-0.461894,-0.191920,18,2,1,5,1,6.831072e-02,0.263086,0
5,5,7,sf100x0,sf100x0O2-6_optsp_a0m2.out,C,0.221689,0.308617,0.215658,-0.092959,0,...,-0.416906,-0.059564,18,2,1,5,1,1.112409e-02,0.263086,0
6,6,8,sf100x0,None,C,-0.138080,-0.337969,-0.357701,-0.019732,0,...,0.389078,-0.188655,18,2,1,5,1,1.082265e-02,0.263086,0
7,7,9,sf100x0,None,C,0.243169,0.054121,-0.032052,-0.086173,0,...,-0.182024,0.039573,18,1,1,5,1,8.823929e-02,0.263086,0
8,8,10,sf100x0,None,C,-0.012210,0.079364,0.079428,0.000064,0,...,-0.220231,-0.121254,18,1,1,5,1,6.307182e-03,0.263086,0
9,9,11,sf100x0,sf100x0O2-10_optsp_a0m2.out,C,-0.006535,0.093478,0.074304,-0.019174,0,...,0.029898,-0.063652,18,1,1,5,1,3.980934e-02,0.263086,0


In [11]:
df_ts = df_ts.assign(random_ordering=np.random.rand(df_ts.shape[0]))

In [13]:
df_ts.iloc[0].random_ordering

0.47516026809140055

In [18]:
l, c = search_for_active_catalysts(df_ts, order_col="random_ordering", feature_cols=feature_cols, find_num=100)
print(len(l))
print(c)

100
603


In [27]:
df_test_all = df_test_all.drop_duplicates()

In [29]:
df_test_all = df_test_all.assign(random_ordering=np.random.rand(df_test_all.shape[0]))

In [31]:
l_O2, c_O2 = search_for_active_catalysts(df_test_all, order_col="random_ordering", feature_cols=feature_cols, find_num=100)
print(len(l_O2), c_O2)
l_t, c_t = search_for_active_catalysts(df_test_all, order_col="predict_proba", feature_cols=feature_cols, find_num=100)
print(len(l_t), c_t)

100 547
100 102
